Tools, helpful functions for development and debugging
====================



In [1]:
from sage.algebras.flag_algebras import *

In [5]:
G = GraphTheory
G.exclude(G(5))
G.optimize(G(2), 5, exact=True)

Base flags generated, their number is 33
The relevant ftypes are constructed, their number is 5
Block sizes before symmetric/asymmetric change is applied: [6, 8, 8, 8, 8]


Done with mult table for Ftype on 3 points with edges=[[0, 1], [0, 2], [1, 2]]: : 5it [00:00, 785.77it/s]

Tables finished
Constraints finished
Running sdp without construction. Used block sizes are [6, 4, 4, 6, 2, 6, 2, 4, 4, -33, -2]


CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -2.0756152e+01 Ad: 7.27e-01 Dobj: -1.7012565e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -2.1130759e+01 Ad: 9.52e-01 Dobj: -4.3177375e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.6757269e+01 Ad: 9.23e-01 Dobj: -4.6196856e-01 
Iter:  4 Ap: 9.89e-01 Pobj: -4.9879585e+00 Ad: 6.80e-01 Dobj: -4.6429854e-01 
Iter:  5 Ap: 5.64e-01 Pobj: -3.9924503e+00 Ad: 9.30e-01 Dobj: -4.7229837e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.2275641e+00 Ad: 7.13e-01 Dobj: -4.8716599e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -9.1340419e-01 Ad: 7.03e-01 Dobj: -5.5747221e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -8.2581439e-01 Ad: 7.40e-01 Dobj: -6.4451512e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -7.7279354e-01 Ad: 7.69e-01 Dobj: -6.9335845e-01 
Iter: 10 Ap: 1.00e+00 Pobj: -7.5637739e-01 Ad: 8.87e-01 Dobj: -7.3189013e-01 
Iter: 11 Ap: 1.00e+00 Pobj: -7.5052711e-01 Ad: 1.00e+00 Dobj: -7.4791845e-01 
Iter: 12 Ap: 9.90e-01 Pobj: -7.5002816e-01 Ad: 1.00e+

{'result': 3/4,
 'X': [
                                                                                                                                                               [  115848148465/57657992704 -115848148465/172973978112 -115848148465/172973978112 -115848148465/172973978112                   -133/512                   -133/512                   -133/512                   -135/256]  [  80074519301/172973978112                       1/32                       1/32  -18792788809/115315985408                      -3/32                      -3/32 -263919710777/345947956224                   -69/1024]  [ 549/1024  -99/1024 -153/1024 -153/1024     3/512     3/512    -7/128     3/256]  [431952967/10810873632                1/1024                1/1024                1/1024                 3/512                 3/512                 3/512 -431952967/3603624544]
                                                                                                                      

In [72]:
sys.setprofile(None)

def tracefunc(frame, event, arg, indent=[0]):
    if event == "call":
        indent[0] += 2
        module_name = inspect.getmodule(frame.f_code).__name__ if inspect.getmodule(frame.f_code) else "unknown"
        class_name = ""
        try:
            class_name = inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True)[-1][0][0].__name__
        except:
            if 'self' in frame.f_locals:
                class_name = str(inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True))
            else:
                class_name = ""
        function_name = frame.f_code.co_name
        tst = module_name + function_name
        if "IPython" in tst or "enum" in tst or "typing" in tst or "traitlets" in tst or "util" in tst or "ipykernel" in tst:
            pass
        else:
            if "unknown" in tst or "tokenize" in tst or "logging" in tst or "json" in tst or "preparse" in tst or indent[0]<=0:
                pass
            else:
                print("-" * indent[0] + "> call function", function_name, "in", class_name, "from", module_name)
    elif event == "return":
        indent[0] -= 2
    return tracefunc

#sys.setprofile(tracefunc)

#asd = sum([G(0)])

In [3]:
#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#for debug printing values with their variable names
def debug(*args):
    import inspect
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = s.split('(')[1].split(')')[0].split(',')
    names = [name.strip() for name in r]
    
    for name, value in zip(names, args):
        if isinstance(value, str):
            print(value)
        else:
            print(f"{name}: {value}")


In [14]:
def verify_solution(self, solution, target_element, target_size, maximize=True, positives=None, construction=None):
    #
    # Checking eigenvalues and positivity constraints
    #
    
    if len(solution)==2 and solution[0] in QQ:
        solution = solution[1]
    if len(solution)==3 and solution[0] in QQ:
        solution = solution[1]
    
    if len(solution[-1])>0 and min(solution[-1])<0:
        print("Solution is not valid!")
        print("Linear constraint's coefficient is negative {}".format(min(solution[-1])))
        return -1

    for ii,X in enumerate(solution[:-1]):
        if min(X.eigenvalues())<0:
            print("Solution is not valid!")
            print("Matrix {} is not semidefinite: {}".format(ii, min(X.eigenvalues())))
            return -1
    
    print("Solution matrices are all semidefinite, linear coefficients are all non-negative")

    #
    # Initial setup
    #
    
    mult = -1 if maximize else 1
    base_flags = self.generate_flags(target_size)
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    if target_element.ftype().size()==0:
        one_vector = vector([1]*len(base_flags))
    else:
        one_vector = (target_element.ftype().project()<<(target_size - target_element.ftype().size())).values()
    
    #
    # Create the types used in the calculation
    #
    
    ftype_data = []
    for fs in range(target_size-2, 1, -2):
        for fl in self.generate_flags(fs):
            ftype = fl.subflag(ftype_points=list(range(fs)))
            ns = (target_size + fs)/2
            ftype_data.append((ns, ftype, target_size))
    ftype_data.sort()
    
    print("Done calculating types relevant for the calculation")
    
    #
    # Create the semidefinite matrix data
    #
    
    table_list = []
    for ii, dat in enumerate(ftype_data):
        ns, ftype, target_size = dat
        #calculate the table
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        if table!=None:
            table_list.append(table)
        print("Done with mult table for {}".format(ftype))

    print("Done calculating semidefinite constraints")
    
    #
    # Create the data from linear constraints
    #

    positives_list_exact = []
    if positives != None:
        for ii, fv in enumerate(positives):
            if isinstance(fv, Flag):
                continue
            nf = fv.size()
            df = target_size + fv.ftype().size() - nf
            mult_table = self.mul_project_table(nf, df, fv.ftype(), ftype_inj=[], target_size=target_size)
            fvvals = fv.values()
            m = matrix(QQ, [vector(fvvals*mat) for mat in mult_table])
            positives_list_exact += list(m.T)
            print("Done with positivity constraint {}".format(ii))
    positives_matrix_exact = matrix(QQ, len(positives_list_exact), len(base_flags), positives_list_exact)
    
    print("Done calculating linear constraints")

    #
    # Calculate the bound the solution provides
    #
    
    slacks = target_vector_exact - positives_matrix_exact.T*solution[-1]
    for ii, table in enumerate(table_list):
        for gg, mat_gg in enumerate(table):
            slacks[gg] -= sum([mat_gg.rows()[jj]*solution[ii][jj] for jj in range(mat_gg.nrows())])
    res = min(slacks)*mult
    
    print("The solution is valid, it proves the bound {}".format(res))
    
    return res

In [15]:
verify_solution(G, res, G(2), 4)

Solution matrices are all semidefinite, linear coefficients are all non-negative
Done calculating types relevant for the calculation
Done with mult table for Ftype on 2 points with edges=[]
Done with mult table for Ftype on 2 points with edges=[[0, 1]]
Done calculating semidefinite constraints
Done calculating linear constraints
The solution is valid, it proves the bound 2/3


2/3

In [2]:
G = GraphTheory
G.exclude(G(3))
G.optimize(G(3, edges=[[0, 1], [0, 2]], ftype=[0, 1]), 4, exact=True, construction=[])

Base flags generated, their number is 7
The relevant ftypes are constructed, their number is 2
Block sizes before symmetric/asymmetric change is applied: [3, 4]


Done with mult table for Ftype on 2 points with edges=[[0, 1]]: : 2it [00:00, 687.70it/s]

Tables finished
Constraints finished
Running sdp without construction. Used block sizes are [2, 1, 3, 1, -7, -2]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -2.4031147e+01 Ad: 6.90e-01 Dobj:  2.6085342e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -2.3311969e+01 Ad: 9.45e-01 Dobj: -1.0244791e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0098255e+01 Ad: 8.61e-01 Dobj: -9.5774050e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -2.8128533e+00 Ad: 7.87e-01 Dobj: -9.7496556e-02 
Iter:  5 Ap: 1.00e+00 Pobj: -5.2363527e-01 Ad: 8.86e-01 Dobj: -1.0118969e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -2.2795982e-01 Ad: 9.03e-01 Dobj: -1.3241207e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.9299418e-01 Ad: 8.27e-01 Dobj: -1.6442902e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -1.7392267e-01 Ad: 8.80e-01 Dobj: -1.7004887e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -1.7178533e-01 Ad: 1.00e+00 Dobj: -1.7148966e-01 
Iter: 10 Ap: 9.96e-01 Pobj: -1.7158265e-01 Ad: 1.00e+00 Dobj: -1.7157022e-01 
Iter: 11 Ap: 9.93e

Rounding X matrices


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 150.82it/s]


Calculating resulting bound


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 132.66it/s]

Rounding errors are [0.00046531961922422746, 2.656989831040235e-06, 0.0005193140193275126, 1.2871523029645004e-06, 0]


11/64

In [3]:
(11/64).n()

0.171875000000000

In [38]:
def _round_sdp_solution_no_phi(self, sdp_result, sdp_data, table_constructor, \
                               constraints_data, denom=1024):
    from tqdm import tqdm
    import numpy as np
    from numpy import linalg as LA
    from sage.functions.other import ceil
    from sage.matrix.special import diagonal_matrix

    #unpack variables

    block_sizes, target_list_exact, mat_inds, mat_vals = sdp_data
    target_vector_exact = vector(target_list_exact)
    flags_num, constraints_vals, positives_list_exact, one_vector = constraints_data
    positives_matrix_exact = matrix(QQ, len(positives_list_exact), flags_num, positives_list_exact)

    one_vector_exact = positives_matrix_exact.rows()[-2]
    positives_matrix_exact = positives_matrix_exact[:-2, :] # remove the equality constraints

    flags_num = -block_sizes[-2] # same as |F_n|

    X_matrices_approx = sdp_result['X'][:-2]
    X_matrices_rounded = []
    print("Rounding X matrices")
    for X in tqdm(X_matrices_approx):

        Xr = _round_matrix(X, method=0, denom=denom)
        Xnp = np.array(Xr)
        eigenvalues, eigenvectors = LA.eig(Xnp)
        emin = min(eigenvalues)
        if emin<0:
            eminr = ceil(-emin*denom)/denom
            Xr = matrix(QQ, Xr) + diagonal_matrix(QQ, [eminr]*len(X), sparse=True)
        X_matrices_rounded.append(Xr)
    X_matrices_flat = [vector(_flatten_matrix(X.rows(), doubled=False)) for X in (X_matrices_rounded)]

    e_vector_approx = sdp_result['X'][-1][:-2]
    e_vector_rounded = vector(_round_list(e_vector_approx, force_pos=True, method=0, denom=denom))
    
    phi_vector_approx = sdp_result['y']
    phi_vector_rounded = vector(_round_list(phi_vector_approx, force_pos=True, method=0, denom=denom))

    slacks = target_vector_exact - positives_matrix_exact.T*e_vector_rounded
    block_index = 0
    print("Calculating resulting bound")
    for params in tqdm(table_constructor.keys()):
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        for gg, morig in enumerate(table):
            for plus_index, base in enumerate(table_constructor[params]):
                block_dim = block_sizes[block_index + plus_index]
                X_flat = X_matrices_flat[block_index + plus_index]
                M = base * morig * base.T
                M_flat_vector_exact = vector(_flatten_matrix(M.rows(), doubled=True))
                slacks[gg] -= M_flat_vector_exact*X_flat
        block_index += len(table_constructor[params])
    result = min([slacks[ii]/oveii for ii, oveii in enumerate(one_vector_exact) if oveii!=0])
    slacks -= result*one_vector_exact
    
    rerrs = []
    for ii in range(len(X_matrices_rounded)):
        Xr = X_matrices_rounded[ii]
        Xo = matrix(sdp_result['X'][ii])
        rerrs.append((Xr-Xo).norm())
    rerrs.append((vector(e_vector_approx) - e_vector_rounded).norm())
    print("Rounding errors are {}".format(rerrs))
    
    return result, X_matrices_rounded, e_vector_rounded, slacks, [phi_vector_rounded]

def _round_sdp_solution_phi(self, sdp_result, sdp_data, table_constructor, \
                        constraints_data, phi_vectors_exact, denom=1024):
    r"""
    Round the SDP results output to get something exact.
    """
    
    #unpack variables
    block_sizes, target_list_exact, mat_inds, mat_vals = sdp_data
    target_vector_exact = vector(target_list_exact)
    flags_num, constraints_vals, positives_list_exact, one_vector = constraints_data
    positives_matrix_exact = matrix(QQ, len(positives_list_exact), flags_num, positives_list_exact)
    
    if len(phi_vectors_exact)==0:
        return None
    phi_vector_exact = phi_vectors_exact[0]

    one_vector_exact = positives_matrix_exact.rows()[-2]
    positives_matrix_exact = positives_matrix_exact[:-2, :] # remove the equality constraints
    
    flags_num = -block_sizes[-2] # same as |F_n|
    
    c_vector_approx = vector(sdp_result['X'][-2]) # dim: |F_n|, c vector, primal slack for flags
    c_vector_rounded = vector(_round_list(c_vector_approx, method=0, denom=denom)) # as above but rounded

    # The F (FF) flag indecies where the c vector is zero/nonzero
    c_zero_inds = [FF for FF, xx in enumerate(c_vector_approx) if (abs(xx)<1e-6 or phi_vector_exact[FF]!=0)]
    c_nonzero_inds = [FF for FF in range(flags_num) if FF not in c_zero_inds]



    positives_num = -block_sizes[-1] - 2 # same as m, number of positive constraints (-2 for the equality)

    phi_pos_vector_exact = positives_matrix_exact*phi_vector_exact # dim: m, witness that phi is positive

    e_vector_approx = vector(sdp_result['X'][-1][:-2]) # dim: m, the e vector, primal slack for positivitives
    e_vector_rounded = vector(_round_list(e_vector_approx, method=0, denom=denom)) # as above but rounded

    # The f (ff) positivity constraints where the e vector is zero/nonzero
    e_zero_inds = [ff for ff, xx in enumerate(e_vector_approx) if (abs(xx)<1e-6 or phi_pos_vector_exact[ff]!=0)]
    e_nonzero_inds = [ff for ff in range(positives_num) if ff not in e_zero_inds]



    bound_exact = target_vector_exact*phi_vector_exact 
    # the constraints for the flags that are exact
    corrected_target_relevant_exact = vector([target_vector_exact[FF] - bound_exact for FF in c_zero_inds])
    # the d^f_F matrix, but only the relevant parts for the rounding
    # so F where c_F = 0 and f where e_f != 0
    positives_matrix_relevant_exact = matrix(QQ, len(e_nonzero_inds), len(c_zero_inds), \
                                             [[positives_matrix_exact[ff][FF] for FF in c_zero_inds] for ff in e_nonzero_inds])
    # the e vector, but only the nonzero entries
    e_nonzero_list_rounded = [e_vector_rounded[ff] for ff in e_nonzero_inds]
    
    
    
    # 
    # Flatten the matrices relevant for the rounding
    # 
    # M table transforms to a matrix, (with nondiagonal entries doubled)
    # only the FF index matrices corresponding with tight constraints are used
    # 
    # X transforms to a vector
    # only the semidefinite blocks are used
    # 

    # The relevant entries of M flattened to a matrix this will be indexed by 
    # c_zero_inds and the triples from the types
    M_flat_relevant_matrix_exact = matrix(QQ, len(c_zero_inds), 0, 0, sparse=True)
    X_flat_vector_rounded = [] # The rounded X values flattened to a list
    block_index = 0
    block_info = []
    for params in table_constructor.keys():
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)

        for plus_index, base in enumerate(table_constructor[params]):
            block_info.append([ftype, base])
            X_approx = sdp_result['X'][block_index + plus_index]
            X_flat_vector_rounded += _round_list(_flatten_matrix(X_approx), method=0, denom=denom)

            M_extra = []

            for FF in c_zero_inds:
                M_FF = table[FF]
                M_extra.append(_flatten_matrix((base * M_FF * base.T).rows(), doubled=True))

            M_flat_relevant_matrix_exact = M_flat_relevant_matrix_exact.augment(matrix(M_extra))
        block_index += len(table_constructor[params])


    # 
    # Append the relevant M matrix and the X with the additional values from
    # the positivity constraints. 
    #
    # Then correct the x vector values
    # 

    M_matrix_final = M_flat_relevant_matrix_exact.augment(positives_matrix_relevant_exact.T)
    x_vector_final = vector(X_flat_vector_rounded+e_nonzero_list_rounded)


    # Correct the values of the x vector, based on the minimal L_2 norm
    x_vector_corr = x_vector_final - M_matrix_final.T * \
    (M_matrix_final * M_matrix_final.T).pseudoinverse() * \
    (M_matrix_final*x_vector_final - corrected_target_relevant_exact) 
    
    #
    # Recover the X matrices and e vector from the corrected x
    #

    e_nonzero_vector_corr = x_vector_corr[-len(e_nonzero_inds):]
    e_vector_corr = vector(QQ, positives_num, dict(zip(e_nonzero_inds, e_nonzero_vector_corr)))
    if len(e_vector_corr)>0 and min(e_vector_corr)<0:
        print("Linear coefficient is negative: {}".format(min(e_vector_corr)))
        return None
    
    X_final = []
    slacks = target_vector_exact - positives_matrix_exact.T*e_vector_corr
    block_index = 0
    for params in table_constructor.keys():
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        for plus_index, base in enumerate(table_constructor[params]):
            block_dim = block_sizes[block_index + plus_index]
            X_ii_small, x_vector_corr = _unflatten_matrix(x_vector_corr, block_dim)
            X_ii_small = matrix(X_ii_small)
            
            # verify semidefiniteness
            if not X_ii_small.is_positive_semidefinite():
                print("Rounded X matrix {} is not semidefinite: {}".format(block_index+plus_index, min(X_ii_small.eigenvalues())))
                return None
            
            # update slacks
            for gg, morig in enumerate(table):
                M = base * morig * base.T
                M_flat_vector_exact = vector(_flatten_matrix(M.rows(), doubled=True))
                slacks[gg] -= M_flat_vector_exact*vector(_flatten_matrix(X_ii_small.rows(), doubled=False))
            
            X_final.append(X_ii_small)
        block_index += len(table_constructor[params])
    
    result = min(slacks)
    slacks -= vector([result]*len(slacks))
    
    return result, X_final, e_vector_corr, slacks, phi_vectors_exact

def _format_optimizer_output(self, table_constructor, mult=1, sdp_output=None, rounding_output=None, file="default"):
    r"""
    Formats the outputs to a nice certificate
    
    The result contains: the final bound, the X matrices, the linear coefficients, the slacks,
                        a guess or exact construction, the list of base flags, the list of used (typed) flags
    """
    
    import json
    
    target_size = 0
    typed_flags = {}
    for params in table_constructor.keys():
        ns, ftype, target_size = params
        safekey = "target:" + str(ns) + "; " + self.flag_compact_repr(ftype)
        typed_flags[safekey] = [self.flag_compact_repr(xx) for xx in self.generate_flags(ns, ftype)]
    
    base_flags = [self.flag_compact_repr(xx) for xx in self.generate_flags(target_size)]
    
    result = None
    X_original = None
    e_vector = None
    slacks = None
    phi_vecs = None
    
    if sdp_output!=None:
        result = sdp_output['primal']
        X_original = [matrix(dat) for dat in sdp_output['X'][:-2]]
        e_vector = vector(sdp_output['X'][-1])
        slacks = vector(sdp_output['X'][-2])
        phi_vecs = [vector(sdp_output['y'])]
    elif rounding_output!=None:
        result, X_original, e_vector, slacks, phi_vecs = rounding_output
    
    result *= mult
    Ds, Ls = self._fix_X_bases(table_constructor, X_original)
    
    cert_dict = {"result": result, 
                 "D vectors": Ds,
                 "L matrices": Ls,
                 "e": e_vector,
                 "slacks": slacks,
                 "phi": phi_vecs,
                 "base flags": base_flags,
                 "typed flags": typed_flags
                }
    return cert_dict

def optimize(self, target_element, target_size, maximize=True, positives=None, \
                     construction=None, file=None, exact=False, denom=1024):
    from csdpy import solve_sdp
    from tqdm import tqdm
    import sys
    import io
    import time

    #
    # Initial setup
    #

    if target_size not in self.sizes():
        raise ValueError("For theory {}, size {} is not allowed.".format(self._name, target_size))

    base_flags = self.generate_flags(target_size)
    print("Base flags generated, their number is {}".format(len(base_flags)))
    mult = -1 if maximize else 1
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    sdp_data = self._target_to_sdp_data(target_vector_exact)
    
    #
    # Create the relevant ftypes
    #
    
    ftype_data = self._get_relevant_ftypes(target_size)
    print("The relevant ftypes are constructed, their number is {}".format(len(ftype_data)))
    flags = [self.generate_flags(dat[0], dat[1]) for dat in ftype_data]
    flag_sizes = [len(xx) for xx in flags]
    print("Block sizes before symmetric/asymmetric change is applied: {}".format(flag_sizes))
    
    #
    # Create the table constructor and add it to sdp_data
    #
    
    table_constructor = self._create_table_constructor(ftype_data, target_size)
    sdp_data = self._tables_to_sdp_data(table_constructor, prev_data=sdp_data)
    print("Tables finished")

    #
    # Add constraints data and add it to sdp_data
    #
    
    constraints_data = self._create_constraints_data(positives, target_element, target_size)
    sdp_data = self._constraints_to_sdp_data(constraints_data, prev_data=sdp_data)
    print("Constraints finished")
    
    #
    # If construction is None or [] then run the optimizer without any construction
    #
    
    if construction==None or construction==[]:
        print("Running sdp without construction. Used block sizes are {}".format(sdp_data[0]))
        
        time.sleep(float(0.1))
        initial_sol = solve_sdp(*sdp_data)
        time.sleep(float(0.1))
        
        # Format the result and return it if floating point values are fine
        if (not exact):
            if file==None:
                return initial_sol['primal'] * mult
            return _format_optimizer_output(self, table_constructor, mult=mult, sdp_output=initial_sol, file=file)
        
        # Guess the construction in this case
        if construction==None:
            one_vector = constraints_data[-1]
            phi_vector_original = initial_sol['y']
            phi_vector_rounded, error_coeff = _round_adaptive(initial_sol['y'], one_vector)
            if error_coeff<1e-6:
                alg = FlagAlgebra(QQ, self)
                construction = alg(target_size, phi_vector_rounded)
                phipr = str(construction)
                print("The initial run gave an accurate looking construction")
                if len(phipr)<1000:
                    print("Rounded construction vector is: \n{}".format(phipr))
            else:
                print("The initial run didn't provide an accurate construction")
                construction = []
        
        # If nothing was provided or the guess failed, then round the current solution
        if construction==[]:
            rounding_output = _round_sdp_solution_no_phi(self, initial_sol, sdp_data, table_constructor, \
                                                             constraints_data, denom=denom)
            if file==None:
                return rounding_output[0] * mult
            return _format_optimizer_output(self, table_constructor, mult=mult, rounding_output=rounding_output, file=file)
    
    
    #
    # Run the optimizer (again if a construction was guessed) with the construction
    #
    
    if isinstance(construction, FlagAlgebraElement):
        phi_vectors_exact = [construction.values()]
    else:
        phi_vectors_exact = [xx.values() for xx in construction]

    #
    # Adjust the table to consider the kernel from y_rounded
    #

    print("Adjusting table with kernels from construction")
    table_constructor = self._adjust_table_phi(table_constructor, phi_vectors_exact)
    sdp_data = self._target_to_sdp_data(target_vector_exact)
    sdp_data = self._tables_to_sdp_data(table_constructor, prev_data=sdp_data)
    sdp_data = self._constraints_to_sdp_data(constraints_data, prev_data=sdp_data)
    
    #
    # Then run the optimizer
    #
    
    print("Running SDP after kernel correction. Used block sizes are {}".format(sdp_data[0]))
    time.sleep(float(0.1))
    final_sol = solve_sdp(*sdp_data)
    time.sleep(float(0.1))

    # Quickly deal with the case when no rounding is needed
    if (not exact):
        if file==None:
            return final_sol['primal'] * mult
        return _format_optimizer_output(self, table_constructor, mult=mult, sdp_output=final_sol, file=file)
    
    
    print("Starting the rounding of the result")
    rounding_output = _round_sdp_solution_phi(self, final_sol, sdp_data, table_constructor, \
                                                                     constraints_data, phi_vectors_exact, denom=denom)
    if rounding_output==None:
        print("Rounding based on construction was unsuccessful")
        rounding_output = _round_sdp_solution_no_phi(self, final_sol, sdp_data, table_constructor, \
                                                        constraints_data, denom=denom)
    
    print("Final rounded bound is {}".format(rounding_output[0]*mult))
    
    if file==None:
        return rounding_output[0] * mult
    return _format_optimizer_output(self, table_constructor, mult=mult, rounding_output=rounding_output, file=file)

In [32]:
def _flatten_matrix(mat, doubled=False):
    r"""
    Flatten a symmetric matrix, optionally double non-diagonal elements
    """
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def _unflatten_matrix(ls, dim, doubled=False):
    r"""
    Unflatten a symmetric matrix, optionally correct for the doubled non-diagonal elements
    """
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

def _round(value, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a number using either 
    method=0 - simple fixed denominator
    method=1 - continued fractions
    """
    if method==0:
        return QQ(round(value*denom)/denom)
    else:
        from sage.rings.continued_fraction import continued_fraction
        cf = continued_fraction(value)
        for ii, xx in enumerate(cf.quotients()):
            if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
                if ii>0:
                    return cf.convergent(ii-1)
                return 0
        return cf.value()

def _round_list(ls, force_pos=False, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a list
    """
    if force_pos:
        return [max(_round(xx, method, quotient_bound, denom_bound, denom), 0) for xx in ls]
    else:
        return [_round(xx, method, quotient_bound, denom_bound, denom) for xx in ls]

def _round_matrix(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a matrix
    """
    return matrix(QQ, [_round_list(xx, False, method, quotient_bound, denom_bound, denom) for xx in mat])

def _round_ldl(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    from sage.matrix.special import diagonal_matrix
    r"""
    Helper function, to round a matrix using ldl decomposition
    """
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, _round_matrix(mat_ldl[1], method, quotient_bound, denom_bound, denom))
    D = diagonal_matrix(QQ, _round_list(mat_ldl[2].diagonal(), True, method, quotient_bound, denom_bound, denom))
    pl = P*L
    return pl*D*pl.T

def _round_adaptive(ls, onevec, denom=1024):
    r"""
    Adaptive rounding based on continued fraction and preserving an inner product
    with `onevec`
    
    If the continued fraction rounding fails fall back to a simple denominator method
    """
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([_round(xx, quotient_bound=resol1, denom_bound=resol2) for xx in orig])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    if best_vec==None:
        rvec = vector(QQ, _round_list(ls, True, method=0, denom=denom))
        best_vec = rvec/(rvec*onevec)
    return best_vec, ((best_vec-orig)/(len(orig)**0.5)).norm()

def _fraction_print(val, thr=20):
    r"""
    Print out a fraction. If it is too big, then print an approximation
    indicating with a ? that it is not an exact value.
    """
    if len(str(val))>thr:
        return str(val.n())+"?"
    else:
        return str(val)